***TUTORIAL 6***

---



Step 1: Import Libraries


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_resnet
import numpy as np

Step 2: Load and Preprocess the Image

In [ ]:
def load_and_preprocess_image(img_path, model_name):
    # Load the image with target size (224, 224)
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to 224x224 pixels
    img_array = image.img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    if model_name == 'vgg':
        # Preprocess image according to the model's requirements
        img_array = preprocess_vgg(img_array)
    elif model_name == 'resnet':
        img_array = preprocess_resnet(img_array)

    return img_array

Step 3: Load Pretrained Models

In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50

vgg_model = VGG16(weights='imagenet')
resnet_model = ResNet50(weights='imagenet')

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Step 4: Make Predictions

In [ ]:
from tensorflow.keras.applications.vgg16 import decode_predictions as decode_vgg
from tensorflow.keras.applications.resnet50 import decode_predictions as decode_resnet

def make_top_5_predictions(model, img_array, model_name):
    # Make predictions
    if model_name == 'vgg':
        preds = model.predict(img_array)
        return decode_vgg(preds, top=5)  # decode predictions (VGG format)
    elif model_name == 'resnet':
        preds = model.predict(img_array)
        return decode_resnet(preds, top=5)  # decode predictions (Resnet format)

def print_top_5_predictions(model, img_path, model_name):
    img_array = load_and_preprocess_image(img_path, model_name)  # Load and preprocess image
    predictions = make_top_5_predictions(model, img_array, model_name)  # get top 5 predictions
    print(f"Top 5 predictions with probabilities for {model_name.upper()}:")
    for i, (imagenet_id, label, prob) in enumerate(predictions[0]):
        print(f"{i+1}. {label} ({prob*100:.2f}% probability)")

Step 5: Define Image Path

In [ ]:
# Define the path to your image
#img_path = r"D:\Deep Learning Course\Codes\Images for pretraining\Dog.jpg"

img_path = r"/cat.png"

Step 6: Get Predictions from VGG16 and ResNet50

In [ ]:
# Predict top 5 using VGG16
vgg_img_array = load_and_preprocess_image(img_path, 'vgg')
vgg_predictions = make_top_5_predictions(vgg_model, vgg_img_array, 'vgg')
print("Top 5 predictions from VGG16:")
for i, pred in enumerate(vgg_predictions[0]):
    print(f"{i+1}. {pred[1]} ({pred[2]*100:.2f}% probability)")

# Predict top 5 using Resnet50
resnet_img_array = load_and_preprocess_image(img_path, 'resnet')
resnet_predictions = make_top_5_predictions(resnet_model, resnet_img_array, 'resnet')
print("Top 5 predictions from ResNet50:")
for i, pred in enumerate(resnet_predictions[0]):
    print(f"{i+1}. {pred[1]} ({pred[2]*100:.2f}% probability)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Top 5 predictions from VGG16:
1. golden_retriever (73.44% probability)
2. otterhound (10.59% probability)
3. Afghan_hound (6.84% probability)
4. kuvasz (2.69% probability)
5. English_setter (1.06% probability)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Top 5 predictions from ResNet50:
1. otterhound (55.41% probability)
2. Afghan_hound (16.62% probability)
3. golden_retriever (11.04% probability)
4. standard_poodle (10.70% probability)
5. bloodhound (1.66% probability)


***TASK 1: Trying Different Architectures***

---



Making AlexNet architecture from scratch while importing ResNet01 & MobileNet.
I gave a cat image to all models as input and the result showed Alexnet & MobileNet probability faster in 0sec while ResNet101 took 4sec. ResNet & MobileNet achieved higher accuracy while Alexnet showed mostly wrong probabilities.


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg, decode_predictions as decode_vgg
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50, decode_predictions as decode_resnet50
from tensorflow.keras.applications.resnet import ResNet101, preprocess_input as preprocess_resnet101, decode_predictions as decode_resnet101
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input as preprocess_mobile, decode_predictions as decode_mobile
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import numpy as np

'''Custom AlexNet model'''
def create_alexnet():
    model = Sequential()
    model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding='valid', activation='relu', input_shape=(224,224,3)))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(5,5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Conv2D(filters=384, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=384, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))  # For ImageNet classes
    return model

#                              Loading and Preprocessing the Image
def load_and_preprocess_image(img_path, model_name):
    target_size = (224, 224) if model_name != 'mobile' else (224, 224)  # MobileNet also 224
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    if model_name == 'vgg':
        img_array = preprocess_vgg(img_array)
    elif model_name in ['resnet', 'resnet101']:
        img_array = preprocess_resnet50(img_array) if model_name == 'resnet' else preprocess_resnet101(img_array)
    elif model_name == 'mobile':
        img_array = preprocess_mobile(img_array)
    elif model_name == 'alex':
        img_array /= 255.0  # Basic normalization for custom AlexNet
    return img_array

#                              Making Predictions
def make_top_5_predictions(model, img_array, model_name):
    preds = model.predict(img_array)
    if model_name == 'vgg':
        return decode_vgg(preds, top=5)
    elif model_name == 'resnet':
        return decode_resnet50(preds, top=5)
    elif model_name == 'resnet101':
        return decode_resnet101(preds, top=5)
    elif model_name == 'mobile':
        return decode_mobile(preds, top=5)
    elif model_name == 'alex':
        return decode_vgg(preds, top=5)  # Reuse VGG decoder as fallback (ImageNet classes)
    return None

def print_top_5_predictions(model, img_path, model_name):
    img_array = load_and_preprocess_image(img_path, model_name)
    predictions = make_top_5_predictions(model, img_array, model_name)
    print(f"Top 5 predictions for {model_name.upper()}:")
    for i, (imagenet_id, label, prob) in enumerate(predictions[0]):
        print(f"{i+1}. {label} ({prob*100:.2f}% probability)")

#                              Loading Pretrained Models
alex_model = create_alexnet()  # Not pretrained; train or load weights separately
resnet101_model = ResNet101(weights='imagenet')
mobile_model = MobileNet(weights='imagenet')

# Define image path
img_path = r"cat.png"  # Or your Colab path

# Get predictions
print_top_5_predictions(alex_model, img_path, 'alex')
print_top_5_predictions(resnet101_model, img_path, 'resnet101')
print_top_5_predictions(mobile_model, img_path, 'mobile')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
Top 5 predictions for ALEX:
1. electric_fan (0.11% probability)
2. cassette_player (0.11% probability)
3. beach_wagon (0.11% probability)
4. rotisserie (0.11% probability)
5. hip (0.11% probability)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Top 5 predictions for RESNET101:
1. tabby (35.45% probability)
2. tiger_cat (19.56% probability)
3. hamper (6.27% probability)
4. printer (2.00% probability)
5. screen (1.65% probability)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 675ms/step
Top 5 predictions for MOBILE:
1. tiger_cat (48.75% probability)
2. tabby (41.32% probability)
3. Egyptian_cat (6.21% probability)
4. window_screen (0.55% probability)
5. Band_Aid (0.34% probability)


***Task 2: Transfer Learning***

---



I uploaded autism kaggle dataset in my drive mounted it and gave its path in code. The results showed both ResNet50 & VGG16 gave 80.67 test accuracy. The total runtime was 21min

In [3]:
# Step 1 Import Libraries
# Similar to tutorial, but added Sequential for building the model and ImageDataGenerator for dataset loading
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_resnet
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

# Step 2 Load and Preprocess the Dataset
# Adapted from single image to batch loading for the entire dataset using generators.
# Assumes preprocessing specific to each model, like in the tutorial.
def create_datagen(preprocess_func):
    return ImageDataGenerator(
        preprocessing_function=preprocess_func,  # Model-specific preprocessing
        rotation_range=20,  # Optional data augmentation for better generalization
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        validation_split=0.0  # If no separate valid folder, you can split here; otherwise, use separate generators
    )

# Setting datast path
data_dir = r'/content/drive/MyDrive/autism_dataset_kaggle'

# Assuming standard structure; adjust if yours differs
test_dir = os.path.join(data_dir, 'test')
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')

batch_size = 32
target_size = (224, 224)  # Matches tutorial's image size

# For VGG16 (uses its preprocess)
train_gen_vgg = create_datagen(preprocess_vgg).flow_from_directory(
    train_dir, target_size=target_size, batch_size=batch_size, class_mode='binary'
)
valid_gen_vgg = create_datagen(preprocess_vgg).flow_from_directory(
    valid_dir, target_size=target_size, batch_size=batch_size, class_mode='binary', shuffle=False
)
test_gen_vgg = create_datagen(preprocess_vgg).flow_from_directory(
    test_dir, target_size=target_size, batch_size=batch_size, class_mode='binary', shuffle=False
)

# For ResNet50 (uses its preprocess)
train_gen_resnet = create_datagen(preprocess_resnet).flow_from_directory(
    train_dir, target_size=target_size, batch_size=batch_size, class_mode='binary'
)
valid_gen_resnet = create_datagen(preprocess_resnet).flow_from_directory(
    valid_dir, target_size=target_size, batch_size=batch_size, class_mode='binary', shuffle=False
)
test_gen_resnet = create_datagen(preprocess_resnet).flow_from_directory(
    test_dir, target_size=target_size, batch_size=batch_size, class_mode='binary', shuffle=False
)

# Print class mapping (0: autistic, 1: non_autistic – check your folder order)
print("Class indices:", train_gen_vgg.class_indices)

# Step 3: Load Pretrained Models
# (Similar to tutorial, but set include_top=False to remove ImageNet classifier head)
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base layers (transfer learning: only train the new head initially)
vgg_base.trainable = False
resnet_base.trainable = False

# Step 4: Build Models with Custom Head for Binary Classification
# (Adapted from prediction function; now we build a full model for training)
def build_model(base_model):
    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),  # To prevent overfitting
        Dense(1, activation='sigmoid')  # Binary output (autistic vs. non_autistic)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

vgg_model = build_model(vgg_base)
resnet_model = build_model(resnet_base)

# Step 5: Define Dataset Paths
# (Already done above; this is where you set data_dir)

# Step 6: Train (Fine-Tune) and Evaluate the Models
# (Adapted from getting predictions; now we train and test instead of just predicting)
epochs = 10  # Adjust as needed; start small to test

print("Training VGG16 model...")
vgg_history = vgg_model.fit(
    train_gen_vgg,
    validation_data=valid_gen_vgg,
    epochs=epochs
)

print("Evaluating VGG16 on test set...")
vgg_test_loss, vgg_test_acc = vgg_model.evaluate(test_gen_vgg)
print(f"VGG16 Test Accuracy: {vgg_test_acc * 100:.2f}%")

print("\nTraining ResNet50 model...")
resnet_history = resnet_model.fit(
    train_gen_resnet,
    validation_data=valid_gen_resnet,
    epochs=epochs
)

print("Evaluating ResNet50 on test set...")
resnet_test_loss, resnet_test_acc = resnet_model.evaluate(test_gen_resnet)
print(f"ResNet50 Test Accuracy: {resnet_test_acc * 100:.2f}%")

# Optional: Interpret results (e.g., predict on a test batch)
# Get a batch from test_gen_vgg
test_images, test_labels = next(test_gen_vgg)
vgg_preds = vgg_model.predict(test_images)
# Print predictions (threshold 0.5 for binary)
print("Sample VGG16 predictions (0: autistic, 1: non_autistic):", (vgg_preds > 0.5).astype(int).flatten())

Found 2536 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 2536 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Class indices: {'autistic': 0, 'non_autistic': 1}
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training VGG16 model...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 626s 8s/step - accuracy: 0.6279 - loss: 5.4979 - val_accuracy: 0.7900 - val_loss: 0.5117
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 45s 556ms/step - accuracy: 0.6888 - loss: 0.6582 - val_accuracy: 0.7800 - val_loss: 0.5203
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 44s 551ms/step - accuracy: 0.7197 - loss: 0.5704 - val_accuracy: 0.7400 - val_loss: 0.4862
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 43s 541ms/step - accuracy: 0.7450 - loss: 0.5570 - val_accuracy: 0.7600 - val_loss: 0.4740
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 46s 569ms/step - accuracy: 0.7486 - loss: 0.5405 - val_accuracy: 0.8000 - val_loss: 0.4480
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 44s 546ms/step - accuracy: 0.7687 - loss: 0.5093 - val_accuracy: 0.8000 - val_loss: 0.4973
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 43s 539ms/step - accuracy: 0.7778 - loss: 0.4663 - val_accuracy: 0.7300 - val_loss: 0.4906
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 43s 538ms/step - accuracy: 0.7657 - loss: 0.4937 - val_accura